In [5]:
import pandas as pd
from pydruid.db import connect
import json
from time import time

In [2]:
with open('test_result.json') as json_file:
    test_result = json.load(json_file)

In [7]:
con = connect(host='localhost', port=8889, path='/druid/v2/sql/', scheme='http')

In [11]:
test_result["timings"]["druid"] = {}
query_index = 1

for query in test_result["queries"]:
    label = f"Q{query_index}"
    print(f"Executing {label}")
    test_result["timings"]["druid"][label] = []

    for i in range(0, 10):
        try:
            start = time()
            pd.read_sql_query(query.replace("InvoiceDate", "__time"), con=con)
            end = time()

            total = end-start
            test_result["timings"]["druid"][label].append(total)
        except:
            test_result["timings"]["druid"][label].append(0)
            print(f"Query {query} not executed.")
            break

    query_index += 1

with open('test_result.json', 'w') as outfile:
    json.dump(test_result, outfile)

Executing Q1
Executing Q2
Executing Q3
Executing Q4
Executing Q5
Executing Q6
Query SELECT CustomerID, Price*Quantity as total_price FROM online_retail WHERE Quantity > 20 ORDER BY total_price DESC not executed.
Executing Q7
